In [3]:
import requests
import json
import pandas as pd
from textblob import TextBlob
import re
import time
import datetime
import numpy as np

## Getting frear greed index data

In [22]:
fgi_page = requests.get('https://api.alternative.me/fng/?limit=10')
fgi = pd.DataFrame(json.loads(fgi_page.content)['data'])
fgi.to_csv('fear_greed_index.csv')

## Crypto Compare api

In [2]:
news_page = requests.get('https://min-api.cryptocompare.com/data/v2/news/?lTs=1522540800&?lang=EN&api_key={610f1e801a5949e3e92544430ae21c7f2b72f9988a37cb6a64aa61566fe6eefd}')
news = pd.DataFrame(json.loads(news_page.content)['Data'])
news.columns


Index(['body', 'categories', 'downvotes', 'guid', 'id', 'imageurl', 'lang',
       'published_on', 'source', 'source_info', 'tags', 'title', 'upvotes',
       'url'],
      dtype='object')

In [45]:
#now i get the full content of body
pd.set_option('display.max_colwidth', -1)
news['body'].head(2)

0    A five-star Swiss hotel, the Dolder Grand, to take BTC payments in May, the Swiss stock exchange launches another crypto-based exchange-traded product On Tuesday, Inacta AG, the tech partner of the top Switzerland’s Dolder Grand...The post Swiss Crypto Hub: Five-Star Hotel to Accept BTC Payments, Stock Exchange SIX Lists XRP-Based ETP appeared first on Ethereum World News.
1    The crypto market is growing, and the bears are losing strength to the point that many speak of a possible change of trend in the coming months. One phenomenon that raised optimism in the community...The post BNB To Reach 100X and XTZ Up To 10X, &#8220;The Don&#8221; Predicts appeared first on Ethereum World News.                                                            
Name: body, dtype: object

In [8]:
for title in news.title:
    print(title)

Cambodia’s Crypto Industry Marches Forward Despite Legal Grey Area
Taking Cyber Security to the Next Level with R3Sec
The Time to Stop Worrying About the Bitcoin Price is Right Now
The Challenges Bitcoin Must Overcome to Become a Global Currency
OmiseGo and Ethereum Founder Donate $1M to GiveDirectly
5 Altcoins with Major Events the week of April 1, 2018 (Gains Likely to Beat Bitcoin Returns!)
New ESMA Measures Impose 2:1 Restriction on Leverage for Crypto CFDs
Bitcoin Price Watch: Bitcoin Endures a Small Rise
IRS Investigations into Cryptocurrency Holders, a Positive Sign
China May Follow Global Cryptocurrency Regulations, Central Bank Report Suggests
Thai Finance Ministry Releases Final Version Of Cryptocurrency Tax Framework
Hacking in Crypto: Cutting-Edge Cybersecurity Measures
Kazakhstan Preparing to Prohibit Crypto Trading and Mining, Says Central Bank
Corsair Campaigns Against Cryptocurrency Mining Rigs in April Fool’s Day Joke
Kleros: The Justice Protocol, Announces Interactive

## CryptoCompare sentiment data and analysis

In [61]:
def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

def get_tweet_sentiment(tweet): 
    ''' 
    Utility function to classify sentiment of passed tweet 
    using textblob's sentiment method 
    '''
    # create TextBlob object of passed tweet text 
    analysis = TextBlob(clean_tweet(tweet)) 
    # set sentiment 
    if analysis.sentiment.polarity > 0: 
        return 'positive'
    elif analysis.sentiment.polarity == 0: 
        return 'neutral'
    else: 
        return 'negative'

def get_tweets(news): 
    ''' 
    Main function to fetch tweets and parse them. 
    '''
    # empty list to store parsed tweets 
    result_news_content = [] 
    news = news

    # parsing tweets one by one 
    for a_news_content,time in zip(news.news_content, news.published_on): 
        # empty dictionary to store required params of a tweet 
        parsed_news_content = {} 

        # saving text of tweet 
        parsed_news_content['full_news'] = a_news_content 
        # saving sentiment of tweet 
        parsed_news_content['sentiment'] = get_tweet_sentiment(a_news_content) 
        parsed_news_content['time'] = time
        result_news_content.append(parsed_news_content)

    return result_news_content 


In [60]:
a = ['a1', 'a2', 'a3']
b = ['b1', 'b2']



# will iterate 2 times,
# the third value of a will not be used
print("Zip:")
for x, y in zip(a, b):
    print(x, y)

# will iterate 6 times,
# it will iterate over each b, for each a
# producing a slightly different outpu
print("List:")
for x, y in [(x,y) for x in a for y in b]:
    print(x, y)

Zip:
a1 b1
a2 b2
List:
a1 b1
a1 b2
a2 b1
a2 b2
a3 b1
a3 b2


## Getting more than 50 news 

In [3]:
def get_data(date):
    """ Query the API for 2000 days historical price data starting from "date". """
    
    api_key = '610f1e801a5949e3e92544430ae21c7f2b72f9988a37cb6a64aa61566fe6eefd'
    url = 'https://min-api.cryptocompare.com/data/v2/news/?lTs={}&?lang=EN&api_key={}'.format(date,api_key)
    news_page = requests.get(url)    
    news_json = news_page.json()
    return news_json


def get_df(from_date, to_date):
    """ Get historical price data between two dates. """
    date = to_date
    holder = []
    # While the earliest date returned is later than the earliest date requested, keep on querying the API
    # and adding the results to a list. 
    while date > from_date:
        data = get_data(date)
        data_df = pd.DataFrame(data['Data'])
        data_df['news_content'] = data_df['title'] + data_df['body']
        holder.append(data_df)
        date = data_df['published_on'].min()
    # Join together all of the API queries in the list.    
    df = pd.concat(holder, axis = 0)                    
    # Remove data points from before from_date
    df = df[df['published_on']>from_date]                       
    # Convert to timestamp to readable date format
    df['time'] = pd.to_datetime(df['published_on'], unit='s')   
    # Make the DataFrame index the time
    df.set_index('time', inplace=True)                  
    # And sort it so its in time order 
    df.sort_index(ascending=False, inplace=True)        
    return df

In [21]:
from_date = int(datetime.date(2017,1,1).strftime("%s")) # 2017 01 01 00:00:00
to_date = int(datetime.date(2019,4,1).strftime("%s")) # 2018 04 01 00:00:00
all_news = get_df(from_date,to_date)


In [22]:
all_news.shape

(89055, 15)

In [24]:
#save all news to csv
all_news.to_csv('all_news.csv', encoding='utf-8', index=False)


In [62]:
df_allllnesw = pd.read_csv('/Users/cx/Desktop/SFU/cmpt733/733project/TA/all_news.csv')
df1 = df_allllnesw

In [63]:
df1.head(2)

,body,categories,downvotes,guid,id,imageurl,lang,published_on,source,source_info,tags,title,upvotes,url,news_content
0,Blockchain technology has been absolutely revo...,Technology|Blockchain,0,http://www.cryptonewsz.com/?p=13460,1004583,https://images.cryptocompare.com/news/cryptone...,EN,1554101992,cryptonewsz,"{'name': 'CryptoNewsZ', 'lang': 'EN', 'img': '...",Blockchain,UNICEF Uses Blockchain Technology to Provide I...,0,https://www.cryptonewsz.com/unicef-uses-blockc...,UNICEF Uses Blockchain Technology to Provide I...
1,Despite the 82 percent drop in the valuation o...,BTC|Trading|Market|Business,0,https://www.ccn.com/?p=172618,1004580,https://images.cryptocompare.com/news/ccn/c204...,EN,1554101991,ccn,"{'name': 'CCN', 'lang': 'EN', 'img': 'https://...",Bitcoin Analysis|Bitcoin Price News|Crypto|New...,"Bitcoin Price Could Blitz Beyond $150,000 Duri...",0,https://www.ccn.com/bitcoin-150000-2023-next-c...,"Bitcoin Price Could Blitz Beyond $150,000 Duri..."


In [43]:
df1 = all_news
df1 = df1.reset_index()
df1.head(2)

NameError: name 'all_news' is not defined

In [47]:
df1 = df1.reset_index()
df1.head(2)

ValueError: cannot insert level_0, already exists

In [64]:
some_news = get_tweets(df1)
print(type(some_news))
some_news[0]
print(len(some_news))


<class 'list'>
89055


In [65]:
some_news[1:10]

[{'full_news': 'Bitcoin Price Could Blitz Beyond $150,000 During Next Bull Cycle in 2023Despite the 82 percent drop in the valuation of the crypto market in the past year and the inability of bitcoin to recover beyond $4,000 in the past three months, one analyst sees the dominant cryptocurrency surpassing $150,000 by 2023. Josh Rager, a cryptocurrency technical analyst and investor, said that based on the price trend of bitcoin since 2011 and the pattern of bitcoin rebounding from a correction following an all-time high, he sees bitcoin achieving a new peak price in 2,051 days, by 2023. The next Bitcoin cycle should peak out in July 2023 and could reach a price The post Bitcoin Price Could Blitz Beyond $150,000 During Next Bull Cycle in 2023 appeared first on CCN',
  'sentiment': 'positive',
  'time': 1554101991},
 {'full_news': 'Dogecoin No Longer Worth 1 Doge1 doge now equals 2 doge thanks to a new hard fork. Dogecoin&#8217;s sole remaining active developer and lifelong devotee, Jack

In [72]:
#some_news = get_tweets(df1)
# picking positive tweets from tweets 
ptnews = [one_news for one_news in some_news if one_news['sentiment'] == 'positive'] 
# percentage of positive tweets 
print("Positive news percentage: {} %".format(100*len(ptnews)/len(some_news))) 
# picking negative tweets from tweets 
ntnews = [one_news for one_news in some_news if one_news['sentiment'] == 'negative'] 
# percentage of negative tweets 
print("Negative news percentage: {} %".format(100*len(ntnews)/len(some_news))) 
# percentage of neutral tweets 
print("Neutral news percentage: {} %".format(100*(len(some_news) - len(ntnews) - len(ptnews))/len(some_news))) 

# printing first 10 positive tweets 
print("\n\nPositive news:") 
for a_news in ptnews[:5]: 
    print(a_news['full_news']) 

# printing first 5 negative tweets 
print("\n\nNegative news:") 
for a_news in ntnews[:5]: 
    print(a_news['full_news']) 

Positive news percentage: 76.68744034585369 %
Negative news percentage: 14.704396159676604 %
Neutral news percentage: 8.608163494469709 %


Positive news:
UNICEF Uses Blockchain Technology to Provide Internet to Every School in KyrgyzstanBlockchain technology has been absolutely revolutionary across industries and sectors. Ever since it was launched way back in 2009, the tech has grown to become one of the best creations of the 21st Century. The innovative tech has now several sectors under its wings, right from agriculture to banking and finance. The technology has created [&#8230;]The post UNICEF Uses Blockchain Technology to Provide Internet to Every School in Kyrgyzstan appeared first on CryptoNewsZ.
Bitcoin Price Could Blitz Beyond $150,000 During Next Bull Cycle in 2023Despite the 82 percent drop in the valuation of the crypto market in the past year and the inability of bitcoin to recover beyond $4,000 in the past three months, one analyst sees the dominant cryptocurrency surpas

<class 'list'>
89055


In [66]:
# pd.DataFrame(data)
news_sentiments_df = pd.DataFrame(some_news)
news_sentiments_df.tail(12)


,full_news,sentiment,time
89043,WAVES Weekly No. 31Last week has seen a number...,positive,1487548800
89044,WAVES Weekly No. 30This week we have several u...,positive,1486944000
89045,WAVES Weekly No. 30This week we have several u...,positive,1486944000
89046,WAVES Weekly No. 29Blockchain Developers Meetu...,positive,1486425600
89047,WAVES Weekly No. 29Blockchain Developers Meetu...,positive,1486425600
89048,"Tidex Aiming to Use Waves ""Matcher"" Technology...",positive,1486339200
89049,"Tidex Aiming to Use Waves ""Matcher"" Technology...",positive,1486339200
89050,WAVES Weekly No. 28A transcript of the recent ...,positive,1485734400
89051,WAVES Weekly No. 28A transcript of the recent ...,positive,1485734400
89052,CORE Radio Interview Transcript with Sasha Iva...,positive,1485475200


In [67]:
news_sentiments_df.columns

Index(['full_news', 'sentiment', 'time'], dtype='object')

## Generating sentiment score

In [23]:
news_sentiments_df = pd.DataFrame(some_news)
# news_sentiments_df.shape
# df1.shape
news_info_with_sentiment = pd.concat([df1, news_sentiments_df], join='inner', axis=1)
news_info_with_sentiment.head(2)
news_info_with_sentiment.shape
df_senti = news_info_with_sentiment[['time', 'published_on', 'news_content','sentiment']]
df_senti = df_senti.sort_values(by='published_on', ascending=True)


KeyError: "['time'] not in index"

In [13]:
df_senti['time'].head(1)

NameError: name 'df_senti' is not defined

In [71]:
df_senti = news_sentiments_df[['time','sentiment']]
df_senti['time_second'] = pd.to_datetime(df_senti['time'], unit='s')
df_senti['day'] = df_senti['time_second'].dt.strftime('%Y-%m-%d')
df_senti['month'] = df_senti['time_second'].dt.strftime('%Y-%m')
# df_senti['time_AM_PM'] = coin_price_time['time_second'].dt.strftime('%Y-%m-%d %r')
df_senti.head(1)
df_senti.to_csv('news_PN_sentiment.csv', encoding='utf-8', index=False)

In [68]:

df_senti['date'] = df_senti['time'].dt.date
df_senti['hour'] = df_senti['time'].dt.hour
df_senti['month'] = df_senti['time'].dt.month
df_senti[['time','date','hour','sentiment','published_on']].head(1)




AttributeError: Can only use .dt accessor with datetimelike values

In [141]:
a = int(1.4)
print(a)


b = pd.Timestamp(1513393355.5, unit='s') 
b.date().format()

1


datetime.date(2017, 12, 16)

In [ ]:
# to_t = coin_return.time[10]
# from_t = to_t - 86400

# holder = []
# for i in range(coin_return.shape[0]):
#     to_t = coin_return.time[i]
#     from_t = to_t - 86400
#     news_interval = news[(news['published_on'] >= from_t) & (news['published_on'] <= to_t)]
#     score = news_interval.mean()[-4:]
#     score['time'] = to_t
#     holder.append(score)
# score_df = pd.concat(holder,axis=1).T

In [176]:
#unix time at 0:0:0 each day
days_unix = [1483228800 + i*86400 for i in range(0,821)]
days_ux = pd.DataFrame(days_unix,columns=['date_unix'])

hours_unix = [1483228800 + i*3600 for i in range(19705)]
hours_ux = pd.DataFrame(hours_unix, columns=['hour_unix'])
print(days_ux.shape)
print(hours_ux.shape)

(821, 1)
(19705, 1)


In [177]:
pd.set_option('display.max_colwidth', -1)

## Generating Daily sentiment score

In [68]:
df_senti.head(2)

,time,published_on,news_content,sentiment
89054,2017-01-07 12:00:00,1483790400,Novoletne zaobljubeZačetek leta je tu in z nji...,positive
89053,2017-01-27 00:00:00,1485475200,CORE Radio Interview Transcript with Sasha Iva...,positive


In [80]:
df_senti['time'].head(1)

89054   2017-01-07 12:00:00
Name: time, dtype: datetime64[ns]

In [94]:
len(df_senti)

89055

In [95]:
df_senti.shape

(89055, 4)

In [1]:
df_senti.head(1)

NameError: name 'df_senti' is not defined

In [ ]:
df_senti.to_csv('all_hourly_social_data.csv', encoding='utf-8', index=False)


## Getting social data from cryptocomapre (2017.1.1 to 2019.4.1)

## Daily social data

In [118]:
def get_daily_social_data(date):
    
    api_key = '610f1e801a5949e3e92544430ae21c7f2b72f9988a37cb6a64aa61566fe6eefd'
    url = 'https://min-api.cryptocompare.com/data/social/coin/histo/day?toTs={}&limit=2000&coinId=1182&api_key={}'.format(date, api_key)
    social_page = requests.get(url)    
    social_json = social_page.json()
    return social_json


def get_daily_social_df(from_date, to_date):
    """ Get historical price data between two dates. """
    date = to_date
    holder = []
    # While the earliest date returned is later than the earliest date requested, keep on querying the API
    # and adding the results to a list. 
    while date > from_date:
        data = get_daily_social_data(date)
        data_df = pd.DataFrame(data['Data'])
        holder.append(data_df)
        date = data_df['time'].min()
        
    # Join together all of the API queries in the list.    
    df = pd.concat(holder, axis = 0)                    
    # Remove data points from before from_date
    df = df[df['time']>from_date]                       
    # Convert to timestamp to readable date format
    df['Date_Time'] = pd.to_datetime(df['time'], unit='s')   
    df = df.sort_values(by='Date_Time', ascending=True)       
    return df

In [119]:
from_date = int(datetime.date(2017,1,1).strftime("%s")) # 2017 01 01 00:00:00
to_date = int(datetime.date(2019,4,1).strftime("%s")) # 2018 04 01 00:00:00
all_daily_social_data = get_daily_social_df(from_date,to_date)



In [120]:
all_daily_social_data.head()

,analysis_page_views,charts_page_views,code_repo_closed_issues,code_repo_closed_pull_issues,code_repo_forks,code_repo_open_issues,code_repo_open_pull_issues,code_repo_stars,code_repo_subscribers,comments,...,reddit_subscribers,time,total_page_views,trades_page_views,twitter_favourites,twitter_followers,twitter_following,twitter_lists,twitter_statuses,Date_Time
1181,0,0,0,0,0,0,0,0,0,0,...,0,1483315200,0,0,0,0,0,0,0,2017-01-02
1182,0,0,0,0,0,0,0,0,0,0,...,0,1483401600,0,0,0,0,0,0,0,2017-01-03
1183,0,0,0,0,0,0,0,0,0,0,...,0,1483488000,0,0,0,0,0,0,0,2017-01-04
1184,0,0,0,0,0,0,0,0,0,0,...,0,1483574400,0,0,0,0,0,0,0,2017-01-05
1185,0,0,0,0,0,0,0,0,0,0,...,0,1483660800,0,0,0,0,0,0,0,2017-01-06


In [104]:
all_daily_social_data.to_csv('all_daily_social_data.csv', encoding='utf-8', index=False)


In [105]:
all_daily_social_data.shape

(820, 34)

## Hourly data

In [106]:
def get_hourly_social_data(date):
    
    api_key = '610f1e801a5949e3e92544430ae21c7f2b72f9988a37cb6a64aa61566fe6eefd'
    url = 'https://min-api.cryptocompare.com/data/social/coin/histo/hour?toTs={}&limit=2000&coinId=1182&api_key={}'.format(date, api_key)
    social_page = requests.get(url)    
    social_json = social_page.json()
    return social_json


def get_hourly_social_df(from_date, to_date):
    """ Get historical price data between two dates. """
    date = to_date
    holder = []
    # While the earliest date returned is later than the earliest date requested, keep on querying the API
    # and adding the results to a list. 
    while date > from_date:
        data = get_hourly_social_data(date)
        data_df = pd.DataFrame(data['Data'])
        holder.append(data_df)
        date = data_df['time'].min()
        
        
    # Join together all of the API queries in the list.    
    df = pd.concat(holder, axis = 0)                    
    # Remove data points from before from_date
    df = df[df['time']>from_date]                       
    # Convert to timestamp to readable date format
    df['Date_Time'] = pd.to_datetime(df['time'], unit='s')   
    df = df.sort_values(by='Date_Time', ascending=True)       
    return df


In [107]:
from_date = int(datetime.date(2017,1,1).strftime("%s")) # 2017 01 01 00:00:00
to_date = int(datetime.date(2019,4,1).strftime("%s")) # 2018 04 01 00:00:00
all_hourly_social_data = get_hourly_social_df(from_date,to_date)


In [108]:
all_hourly_social_data.to_csv('all_hourly_social_data.csv', encoding='utf-8', index=False)


In [187]:

x = np.array([[[1,2,3], [4,5,6]],[[1,2,3], [4,5,6]]])
x

array([[[1, 2, 3],
        [4, 5, 6]],

       [[1, 2, 3],
        [4, 5, 6]]])

In [188]:
x.shape

(2, 2, 3)

In [192]:
X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]
x = np.array(X)
x.shape
len(x)

3